In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Cargar datos
df_alpha_beta = pd.read_csv('dataset_alpha_betha.csv')
df_to_predict = pd.read_csv('to_predict.csv')
df_demand = pd.read_csv('dataset_demand_acumulate.csv')

# Preprocesamiento de datos
def preprocesar_datos(df):
    # Codificar variables categóricas
    categoricas = ['Partner', 'Dependents', 'Service1', 'Service2', 
                   'Security', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'Contract', 'PaperlessBilling', 
                   'PaymentMethod']
    
    le = LabelEncoder()
    for col in categoricas:
        df[col] = le.fit_transform(df[col].astype(str))
    
    return df

# Preparar datos de entrenamiento
X_train = preprocesar_datos(df_alpha_beta.drop(['Class', 'autoID'], axis=1))
y_train = df_alpha_beta['Class']

# Preparar datos a predecir
X_predict = preprocesar_datos(df_to_predict.drop(['autoID', 'Demand', 'Class'], axis=1))

# Preparación del preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), X_train.select_dtypes(include=['int64', 'float64']).columns),
    ])

# Pipeline de modelo
modelo = Pipeline([
    ('preprocessor', preprocessor),
    ('clasificador', RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        class_weight='balanced'
    ))
])

# Entrenamiento
modelo.fit(X_train, y_train)

# Evaluación
predicciones_train = modelo.predict(X_train)
reporte = classification_report(y_train, predicciones_train)

# Predicción para datos a predecir
predicciones_finales = modelo.predict(X_predict)

# Guardar modelo
joblib.dump(modelo, 'modelo_argos.joblib')

# Guardar métricas
with open('metricas_modelo.txt', 'w') as f:
    f.write("Modelo de Clasificación Cementos Argos\n")
    f.write(f"Modelo: RandomForestClassifier\n")
    f.write(f"Total Datos Entrenamiento: {len(X_train)}\n")
    f.write("\nReporte Clasificación:\n")
    f.write(reporte)

# Preparar resultados
resultados = df_to_predict.copy()
resultados['Prediccion'] = predicciones_finales
resultados.to_csv('predicciones_finales.csv', index=False)

print("Modelo entrenado y predicciones generadas exitosamente.")

Modelo entrenado y predicciones generadas exitosamente.
